In [1]:
# pip install earthaccess

In [2]:
# pip install xarray

In [9]:
import earthaccess
import xarray as xr

# Authenticate
earthaccess.login()

In [10]:
# Search for datasets
no2_results = earthaccess.search_data(
    short_name='TEMPO_NO2_L3',
    cloud_hosted=True,
    count=5
)

hcho_results = earthaccess.search_data(
    short_name='TEMPO_HCHO_L3', 
    cloud_hosted=True,
    count=5
)

ozone_results = earthaccess.search_data(
    short_name='TEMPO_O3TOT_L3',  # Total ozone
    cloud_hosted=True,
    count=5
)

radiance_results = earthaccess.search_data(
    short_name='TEMPO_RAD_L1',
    cloud_hosted=True,
    count=5
)


In [5]:
# !pip install earthaccess xarray h5netcdf h5py netcdf4 pandas matplotlib


In [6]:
# pip install --upgrade h5netcdf h5py

In [7]:
import pandas as pd

# Simple subset extraction - just basic info from the first few results
file_data = []

# Extract basic info from just the NO2 results (first dataset)
print("🌍 TEMPO NO2 Data Files (Subset)")
print("=" * 40)

if 'no2_results' in locals() and no2_results:
    for i, result in enumerate(no2_results[:3]):  # Only first 3 files
        file_info = {
            'File_Number': i + 1,
            'Dataset': 'TEMPO_NO2_L3',
            'Title': result.get('title', 'N/A')[:50] + '...' if len(str(result.get('title', 'N/A'))) > 50 else result.get('title', 'N/A'),
            'Size_MB': round(result.get('file_size', 0) / (1024*1024), 2) if result.get('file_size') else 'Unknown'
        }
        file_data.append(file_info)
    
    # Create and display DataFrame
    df = pd.DataFrame(file_data)
    print(f"Found {len(no2_results)} total files, showing first {len(df)}:")
    print()
    print(df.to_string(index=False))
    
else:
    print("❌ No NO2 results available. Please run the search cell first.")
    print("Available variables:", [var for var in locals().keys() if not var.startswith('_')])

🌍 TEMPO NO2 Data Files (Subset)
Found 5 total files, showing first 3:

 File_Number      Dataset Title Size_MB
           1 TEMPO_NO2_L3   N/A Unknown
           2 TEMPO_NO2_L3   N/A Unknown
           3 TEMPO_NO2_L3   N/A Unknown


In [8]:
# import pandas as pd
# from datetime import datetime

# # Create a list to store file information
# file_data = []

# # Helper function to extract file information
# def extract_file_info(results, dataset_name):
#     for result in results:
#         file_info = {
#             'Dataset': dataset_name,
#             'Filename': result.get('title', 'N/A'),
#             'Size_MB': round(result.get('file_size', 0) / (1024*1024), 2) if result.get('file_size') else 'N/A',
#             'Date': result.get('time_start', 'N/A'),
#             'Provider': result.get('data_center', 'N/A'),
#             'Format': result.get('format', 'N/A'),
#             'Cloud_Hosted': result.get('cloud_hosted', False)
#         }
#         file_data.append(file_info)

# # Extract information from all search results
# extract_file_info(no2_results, 'TEMPO_NO2_L3')
# extract_file_info(hcho_results, 'TEMPO_HCHO_L3') 
# extract_file_info(ozone_results, 'TEMPO_O3TOT_L3')
# extract_file_info(radiance_results, 'TEMPO_RAD_L1')

# # Create DataFrame
# df = pd.DataFrame(file_data)

# # Display the DataFrame
# print("🌍 TEMPO Satellite Data Files")
# print("=" * 60)
# print(f"Total files found: {len(df)}")
# print("\n📊 File Information DataFrame:")
# print(df.to_string(index=False, max_colwidth=50))

# # Show summary by dataset
# print("\n� Summary by Dataset:")
# summary = df.groupby('Dataset').agg({
#     'Filename': 'count',
#     'Size_MB': lambda x: f"{x.sum():.1f}" if pd.notna(x).any() else 'N/A'
# }).rename(columns={'Filename': 'File_Count', 'Size_MB': 'Total_Size_MB'})
# print(summary.to_string())